API safe request rate is limit 1 request per second.

In [1]:
!pip install requests
!pip install diskcache

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [13]:
import requests
import base64
import json
import os
from datetime import datetime
import pytz
import diskcache
import urllib.parse
from matplotlib import pyplot as plt

In [14]:
project_dir = os.path.abspath('')

API_version = 'v9'
prefix = f"https://api.track.toggl.com/api/"

API = {
    "time_entries" :          prefix + "v8" + "/time_entries",
    "get_current_time_entry": prefix + API_version + "/me/time_entries/current", 
    "me_info" :               prefix + API_version + "/me",
    "projects_me" :           prefix + API_version + "/me/projects",
    "projects_ws" :           prefix + API_version + "/workspaces/4143224/projects",
    "workspaces" :            prefix + API_version + "/workspaces",
}

with open(f"{project_dir}/api_key.txt") as f:
    API_token = f"{f.read()}:api_token"
API_token_base64 = base64.b64encode(API_token.encode('ascii')).decode('ascii')

In [15]:
project_id_names = diskcache.Cache(f"{project_dir}/project_id_names.cache")
project_id_names.clear()

8

In [16]:
def fetch(url, headers={}):
    headers={"Content-Type": "application/json", 
                                    "Host": "api.track.toggl.com",
                                    "authorization": f"Basic {API_token_base64}", 
                                    **headers}
    r = requests.get(url, headers=headers)
    return json.loads(r.content.decode('utf-8'))

In [17]:
def fetch_project(project_id):
    return fetch(f"{API['projects_ws']}/{project_id}")

In [18]:
def filter_dict(d, keys):
    return list(map(lambda x: {k: v for k, v in x.items() if k in keys}, d))

In [19]:
def get_time_entries(start_date, end_date):
    date_to_str = lambda date: date.strftime('%Y-%m-%dT%H:%M:%S') + f'{date.strftime("%z")[:3]}:{date.strftime("%z")[3:]}'
    url_exts = '?'
    url_exts += f"start_date={urllib.parse.quote(date_to_str(start_date))}"
    url_exts += f"&end_date={urllib.parse.quote(date_to_str(end_date))}"
    time_entries = fetch(API['time_entries'] + url_exts)
        
    for time_entry in time_entries:
        print(time_entries)
        project_id = time_entry['pid']
        if project_id not in project_id_names:
            project_id_names[project_id] = fetch_project(project_id)
        time_entry['project_name'] = project_id_names[project_id]['name']
        
    return time_entries

In [20]:
def update_current_time_entry_duration(time_entry):
    time_entry["duration"] = (datetime.utcnow().replace(tzinfo=pytz.utc) - \
                              datetime.fromisoformat(time_entry["start"])).seconds
    return time_entry

def get_day_entries():
    pass

def get_week_entries():
    pass

def get_month_entries():
    pass

def get_last_n_days():
    pass

In [ ]:
start_date = datetime.fromisoformat('2022-11-10T01:00:00+00:00')
end_date = datetime.fromisoformat('2022-11-10T23:59:59+00:00')
get_time_entries(start_date, end_date)

[{'id': 2723808956, 'guid': 'c8fd6d381a112a7d2d1408de530f86ab', 'wid': 4143224, 'pid': 186239363, 'billable': False, 'start': '2022-11-10T01:10:39+00:00', 'stop': '2022-11-10T01:28:29+00:00', 'duration': 1070, 'duronly': False, 'at': '2022-11-10T01:28:30+00:00', 'uid': 5591118}, {'id': 2723821046, 'guid': '95a48921ad86c9df794fc31b6f39a7db', 'wid': 4143224, 'pid': 186119188, 'billable': False, 'start': '2022-11-10T01:33:00+00:00', 'stop': '2022-11-10T01:38:40+00:00', 'duration': 340, 'duronly': False, 'at': '2022-11-10T01:38:42+00:00', 'uid': 5591118}, {'id': 2723827884, 'guid': '1e7ad3e75790067bd2e9370ba249b5fe', 'wid': 4143224, 'pid': 163125866, 'billable': False, 'start': '2022-11-10T02:51:00+00:00', 'stop': '2022-11-10T12:51:00+00:00', 'duration': 36000, 'duronly': False, 'at': '2022-11-10T12:52:17+00:00', 'uid': 5591118}, {'id': 2724555632, 'guid': '13c344375b101b3778b192e18cdbace6', 'wid': 4143224, 'pid': 185725569, 'billable': False, 'start': '2022-11-10T11:51:00+00:00', 'stop': 